In [115]:
import os
os.getcwd()

import functions
print(functions.__file__)
%reload_ext autoreload
%autoreload 2
import importlib, functions
importlib.reload(functions)
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest
#~retencao_pedido_1=analisar_retencao(df_1)
from functions import retidos, analisar_retencao,calcula_viabilidade

/Users/maceli/ifood_cs/code/functions.py


In [77]:
df = pd.read_parquet("../dados/gold/df_decil.parquet")
df.head()


,customer_id,is_target,decil_1,decil_12,num_pedidos_mes_1,num_pedidos_mes_12,total_amount_mes_1,total_amount_mes_12
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,decil 10,decil 7,8,2,273.00,100.00
1,000021924bf8192f64a7cc17ce7c0bc511cba907308f80...,target,decil 9,decil 8,3,2,173.90,119.90
2,000022ced2f3aa411b66ac41167b272ae528c4acdffeeb...,target,decil 6,decil 6,2,2,68.65,68.65
3,000032b594523c3f8868edee4f1577b157e115cd01ab31...,control,decil 7,decil 7,1,1,88.80,88.80
4,0000bb10fb47a1d6b2d73754ef383950ef536c77d07212...,control,decil 9,decil 9,2,2,188.19,188.19


In [78]:
df['decil_1_num'] = df['decil_1'].str.extract('(\d+)').astype(int)
df['decil_12_num'] = df['decil_12'].str.extract('(\d+)').astype(int)
df['delta_decil'] = df['decil_12_num'] - df['decil_1_num']

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/s5/dkrzm08d6wl79n0by4mcn72m0000gn/T/ipykernel_5660/3781260405.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['decil_1_num'] = df['decil_1'].str.extract('(\d+)').astype(int)
/var/folders/s5/dkrzm08d6wl79n0by4mcn72m0000gn/T/ipykernel_5660/3781260405.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['decil_12_num'] = df['decil_12'].str.extract('(\d+)').astype(int)


In [88]:
def classifica_mobilidade(delta):
    if delta == 0:
        return "Estável"
    elif abs(delta) == 1:
        return "Mobilidade Moderada"
    else:
        return "Alta Mobilidade"

df['segmento_mobilidade'] = df['delta_decil'].apply(classifica_mobilidade)

In [89]:
def classifica_direcao(delta):
    if delta < 0:
        return "Upgrade"
    elif delta > 0:
        return "Downgrade"
    else:
        return "Estável"

df['direcao'] = df['delta_decil'].apply(classifica_direcao)


In [84]:
tabela_segmento = (
    df
    .groupby(['segmento_mobilidade', 'direcao', 'is_target'])
    .agg(
        clientes=('customer_id', 'nunique'),
        decil_inicial_medio=('decil_1_num', 'mean'),
        decil_final_medio=('decil_12_num', 'mean'),
        pedidos_medios_mes_1=('num_pedidos_mes_1', 'mean'),
        pedidos_medios_mes_12=('num_pedidos_mes_12', 'mean'),
        ticket_medio_mes_1=('total_amount_mes_1', 'mean'),
        ticket_medio_mes_12=('total_amount_mes_12', 'mean')
    )
    .reset_index()
)




In [86]:
tabela_segmento

,segmento_mobilidade,direcao,is_target,clientes,decil_inicial_medio,decil_final_medio,pedidos_medios_mes_1,pedidos_medios_mes_12,ticket_medio_mes_1,ticket_medio_mes_12
0,Alta Mobilidade,Upgrade,control,63005,8.041219,4.558289,4.109864,1.602587,168.126372,59.897590
1,Alta Mobilidade,Upgrade,target,94674,8.024495,4.534804,4.086412,1.596341,167.860583,59.595718
2,Estável,Estável,control,152564,5.224083,5.224083,2.859777,2.170997,152.255066,112.578641
3,Estável,Estável,target,179684,5.664121,5.664121,3.353203,2.478011,179.177332,128.823753
4,Mobilidade Moderada,Upgrade,control,25888,8.984433,7.984433,5.780632,3.185994,260.700879,142.219690
5,Mobilidade Moderada,Upgrade,target,38561,8.973393,7.973393,5.833874,3.200202,261.626572,141.997371


In [ ]:
# 1. Criar a categoria baseada em segmento_mobilidade + direcao
mapa_categorias = {
    ('Alta Mobilidade', 'Upgrade'): 1,
    ('Estável', 'Estável'): 2,
    ('Mobilidade Moderada', 'Upgrade'): 3
}

df['categoria_segmento'] = df.apply(
    lambda x: mapa_categorias.get((x['segmento_mobilidade'], x['direcao']), None),
    axis=1
)

# 2. Ver quantos clientes tem em cada categoria por grupo (control/target)
resumo_categorias = (
    df
    .groupby(['categoria_segmento', 'is_target'])['customer_id']
    .nunique()
    .reset_index(name='num_clientes')
    .sort_values(['categoria_segmento', 'is_target'])
)




   categoria_segmento is_target  num_clientes
0                   1   control         63005
1                   1    target         94674
2                   2   control        152564
3                   2    target        179684
4                   3   control         25888
5                   3    target         38561


In [99]:
df.columns

Index(['customer_id', 'is_target', 'decil_1', 'decil_12', 'num_pedidos_mes_1',
       'num_pedidos_mes_12', 'total_amount_mes_1', 'total_amount_mes_12',
       'decil_1_num', 'decil_12_num', 'delta_decil', 'segmento_mobilidade',
       'direcao', 'categoria_segmento'],
      dtype='object')

In [108]:
df11=df[df['categoria_segmento']==1]
df12=df[df['categoria_segmento']==2]
df13=df[df['categoria_segmento']==3]

In [105]:
# df com colunas:
# ['customer_id', 'is_target', 'decil_1', 'decil_12',
#  'num_pedidos_mes_1', 'num_pedidos_mes_12',
#  'total_amount_mes_1', 'total_amount_mes_12',
#  'decil_1_num', 'decil_12_num', 'delta_decil',
#  'segmento_mobilidade', 'direcao', 'categoria_segmento']

def retidos11(df, mes0=1, mes1=12, pedidos=1, por_segmento=False):
    """
    Calcula retenção A/B a partir da base wide já agregada por cliente.

    Parâmetros
    ----------
    df : DataFrame
        Base wide por cliente (1 linha por customer_id).
    mes0 : int
        Mês base (ex.: 1).
    mes1 : int
        Mês de comparação (ex.: 12).
    pedidos : int
        Número mínimo de pedidos no mes1 para considerar retido.
        Ex.: 0  -> pelo menos 1 pedido
             1  -> pelo menos 2 pedidos
    por_segmento : bool
        Se True, devolve também resumo por segmento (categoria_segmento /
        segmento_mobilidade / direcao).

    Retorno
    -------
    base : DataFrame
        Base de clientes ativos no mes0 com coluna 'retido'.
    resumo_global : DataFrame
        Resumo por is_target (control x target).
    resumo_segmento : DataFrame (opcional)
        Resumo por segmento + is_target (se por_segmento=True).
    """

    # nomes das colunas de frequência e GMV dinamicamente
    freq0_col = f"num_pedidos_mes_{mes0}"
    freq1_col = f"num_pedidos_mes_{mes1}"
    gmv0_col = f"total_amount_mes_{mes0}"
    gmv1_col = f"total_amount_mes_{mes1}"

    # garante que as colunas existem
    for c in [freq0_col, freq1_col, gmv0_col, gmv1_col]:
        if c not in df.columns:
            raise ValueError(f"Coluna '{c}' não encontrada no DataFrame.")

    base = df.copy()

    # deltas de frequência e GMV
    base["delta_freq"] = base[freq1_col] - base[freq0_col]
    base["delta_gmv"] = base[gmv1_col] - base[gmv0_col]

    # base = clientes ativos no mes0
    base = base[base[freq0_col] > 0].copy()

    # retido = fez mais que `pedidos` no mes1
    # pedidos=0 => >0 pedidos, pedidos=1 => >1 pedidos, etc.
    base["retido"] = (base[freq1_col] > pedidos).astype(int)

    # resumo global A/B
    resumo_global = (
        base.groupby("is_target")
            .agg(
                retidos=("retido", "sum"),
                base=("retido", "count")
            )
    )
    resumo_global["taxa_retencao"] = resumo_global["retidos"] / resumo_global["base"]

    if not por_segmento:
        return base, resumo_global

    # resumo por segmento (categoria + mobilidade + direção)
    resumo_segmento = (
        base
        .groupby(["categoria_segmento", "segmento_mobilidade", "direcao", "is_target"])
        .agg(
            retidos=("retido", "sum"),
            base=("retido", "count")
        )
        .reset_index()
    )
    resumo_segmento["taxa_retencao"] = resumo_segmento["retidos"] / resumo_segmento["base"]

    return base, resumo_global, resumo_segmento


In [114]:
retencao_pedido_1=analisar_retencao(df_1)
retencao_pedido_1

{'target_retencao': 0.456986,
 'control_retencao': 0.360301,
 'lift_absoluto': 0.096684,
 'lift_relativo_percent': 26.83,
 'target_success': 82113,
 'target_total': 179684,
 'control_success': 54969,
 'control_total': 152564,
 'z_stat': 56.412558,
 'p_value': 0.0,
 'significativo': True}

In [113]:
df_1,clientes_retidos_1=retidos11(df11, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
df_1,clientes_retidos_1=retidos11(df12, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
#df_1,clientes_retidos_1=retidos11(df13, mes0=12, mes1=1,pedidos=1)
#print(clientes_retidos_1)

           retidos   base  taxa_retencao
is_target                               
control      63005  63005            1.0
target       94674  94674            1.0
           retidos    base  taxa_retencao
is_target                                
control      54969  152564       0.360301
target       82113  179684       0.456986


In [117]:
def calcula_viabilidade_wide(df,
                             mes_campanha=12,
                             mes_seguinte=1,
                             coupon_value=10.0,
                             margin_rate=0.12):
    """
    Calcula viabilidade econômica (ROI) da campanha usando base WIDE já agregada por cliente.

    Parâmetros
    ----------
    df : DataFrame
        Deve conter, no mínimo:
        - customer_id
        - is_target  ('target' / 'control')
        - num_pedidos_mes_<mes_campanha>
        - num_pedidos_mes_<mes_seguinte>
        - total_amount_mes_<mes_campanha>
        - total_amount_mes_<mes_seguinte>

        Exemplo de colunas para mes_campanha=12 e mes_seguinte=1:
        - num_pedidos_mes_12, num_pedidos_mes_1
        - total_amount_mes_12, total_amount_mes_1

    mes_campanha : int
        Mês em que o cupom foi disponibilizado (ex: 12).
    mes_seguinte : int
        Mês seguinte para retenção (ex: 1).
    coupon_value : float
        Valor médio do cupom (R$).
    margin_rate : float
        Margem líquida sobre o GMV (0.12 = 12%).

    Retorno
    -------
    resultados : dict
        Métricas agregadas de incremento, margem e ROI.
    agg : DataFrame
        Resumo por is_target (pedidos, gmv, clientes) para cada mês.
    """

    # nomes das colunas dinamicamente
    freq_camp_col = f"num_pedidos_mes_{mes_campanha}"
    freq_seg_col  = f"num_pedidos_mes_{mes_seguinte}"
    gmv_camp_col  = f"total_amount_mes_{mes_campanha}"
    gmv_seg_col   = f"total_amount_mes_{mes_seguinte}"

    # checagem básica de colunas
    for c in [freq_camp_col, freq_seg_col, gmv_camp_col, gmv_seg_col]:
        if c not in df.columns:
            raise ValueError(f"Coluna obrigatória '{c}' não encontrada no DataFrame.")

    # agrega por grupo (target/control) para cada mês
    agg = (
        df.groupby("is_target")
          .agg(
              pedidos_campanha=(freq_camp_col, "sum"),
              gmv_campanha=(gmv_camp_col, "sum"),
              pedidos_seguinte=(freq_seg_col, "sum"),
              gmv_seguinte=(gmv_seg_col, "sum"),
              clientes=("customer_id", "nunique")
          )
    )

    # helper para pegar linha por grupo
    row_t = agg.loc["target"]
    row_c = agg.loc["control"]

    # --- Pedidos por cliente em cada mês ---
    pedidos_cli_t_dec = row_t["pedidos_campanha"]   / row_t["clientes"]
    pedidos_cli_c_dec = row_c["pedidos_campanha"]   / row_c["clientes"]

    pedidos_cli_t_jan = row_t["pedidos_seguinte"]   / row_t["clientes"]
    pedidos_cli_c_jan = row_c["pedidos_seguinte"]   / row_c["clientes"]

    # --- GMV por cliente em cada mês ---
    gmv_cli_t_dec = row_t["gmv_campanha"] / row_t["clientes"]
    gmv_cli_c_dec = row_c["gmv_campanha"] / row_c["clientes"]

    gmv_cli_t_jan = row_t["gmv_seguinte"] / row_t["clientes"]
    gmv_cli_c_jan = row_c["gmv_seguinte"] / row_c["clientes"]

    # --- Incremento de pedidos (Target vs Controle) ---
    # Escalando pelo número de clientes TARGET (mesma lógica da função original)
    inc_pedidos_dec = (pedidos_cli_t_dec - pedidos_cli_c_dec) * row_t["clientes"]
    inc_pedidos_jan = (pedidos_cli_t_jan - pedidos_cli_c_jan) * row_t["clientes"]
    inc_pedidos_total = inc_pedidos_dec + inc_pedidos_jan

    # --- Incremento de GMV ---
    inc_gmv_dec = (gmv_cli_t_dec - gmv_cli_c_dec) * row_t["clientes"]
    inc_gmv_jan = (gmv_cli_t_jan - gmv_cli_c_jan) * row_t["clientes"]
    inc_gmv_total = inc_gmv_dec + inc_gmv_jan

    # --- Margem incremental ---
    margem_incremental = inc_gmv_total * margin_rate

    # --- Custo da campanha ---
    # premissa: cupom usado 1x por cliente target que fez pedido no mes_campanha
    base_target_camp = df[
        (df["is_target"] == "target") &
        (df[freq_camp_col] > 0)
    ]["customer_id"].nunique()

    custo_campanha = base_target_camp * coupon_value

    # --- ROI ---
    lucro_incremental = margem_incremental - custo_campanha
    roi = lucro_incremental / custo_campanha if custo_campanha > 0 else None

    resultados = {
        "pedidos_por_cliente_camp_control": pedidos_cli_c_dec,
        "pedidos_por_cliente_camp_target": pedidos_cli_t_dec,
        "pedidos_por_cliente_seg_control": pedidos_cli_c_jan,
        "pedidos_por_cliente_seg_target": pedidos_cli_t_jan,
        "inc_pedidos_camp": inc_pedidos_dec,
        "inc_pedidos_seg": inc_pedidos_jan,
        "inc_pedidos_total": inc_pedidos_total,
        "inc_gmv_camp": inc_gmv_dec,
        "inc_gmv_seg": inc_gmv_jan,
        "inc_gmv_total": inc_gmv_total,
        "margem_incremental": margem_incremental,
        "base_target_camp": base_target_camp,
        "custo_campanha": custo_campanha,
        "lucro_incremental": lucro_incremental,
        "roi": roi,
        "coupon_value": coupon_value,
        "margin_rate": margin_rate,
        "mes_campanha": mes_campanha,
        "mes_seguinte": mes_seguinte,
    }

    return resultados, agg


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [119]:
resultados, agg = calcula_viabilidade_wide(df_1,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 2.1709970897459425, 'pedidos_por_cliente_camp_target': 2.4780113977872267, 'pedidos_por_cliente_seg_control': 2.8597768805222725, 'pedidos_por_cliente_seg_target': 3.3532034015271255, 'inc_pedidos_camp': 55165.5589260901, 'inc_pedidos_seg': 88660.851000236, 'inc_pedidos_total': 143826.4099263261, 'inc_gmv_camp': 2918986.7852843422, 'inc_gmv_seg': 4837500.541719672, 'inc_gmv_total': 7756487.3270040145, 'margem_incremental': 930778.4792404817, 'base_target_camp': 179684, 'custo_campanha': 1796840.0, 'lucro_incremental': -866061.5207595183, 'roi': -0.4819914520822768, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control              331216   17175447.79            436299   23228641.88   
target               445259   23147567.32            602517   32195299.81   

